In [1]:
from dis import dis
import os
import json
import datetime
import csv
from csv import writer
import random
from turtle import distance
import numpy
from Levenshtein import *

In [2]:
os.chdir('G:\My Drive\Term 4.2\IS706 - Software Mining and Analysis\Project')
with open('allpairs.csv', newline='') as csvfile:
    raw_pos_pairs = list(csv.reader(csvfile))

os.chdir('G:\My Drive\Term 4.2\IS706 - Software Mining and Analysis\Project\extracted')
extracted = os.listdir()
filt = list(filter(lambda x: x[1] + '-' + x[3] + '.jar.json' in extracted, raw_pos_pairs)) #select all json files

In [3]:
start = datetime.datetime.now()
print("Start time:", start)

notfound = 0
pcnt = 0

pos_pairs = []
neg_pairs = []

# divider = "__XX__"
divider = ""

for p in filt:
    name = p[1] + '-' + p[3] + '.jar.json'
    namepath = 'G:\\My Drive\\Term 4.2\\IS706 - Software Mining and Analysis\\Project\\extracted\\' + name

    print(pcnt, '/', len(raw_pos_pairs), "Processing:", name)
    print("Searching:", p[6])
    
    f = open(namepath)
    loaded = json.load(f)
    f.close()
    # allcandidates = list(filter(lambda x: x[0] == p[4], loaded[name[:-5]])) #filter candidates with the same return type
    allcandidates = list(filter(lambda x: x[0] == x[0], loaded[name[:-5]]))

    #for this file
    can_pairs = []

    #find out the positive pair
    positive_position = 0
    found = False

    if '(' not in p[6]:
        print("Replacement", p[6], "is not method!!!")
        notfound = 1 + notfound
    else:
        for can in allcandidates:
            
            e_cargList = []

            #check if positive candidate
            if can[1][: can[1].index('(')] == p[6][: p[6].index('(')]:

                #extract args from candidate
                cargs = can[1][can[1].index('(') + 1: can[1].index(')')]
                cargList = cargs.split(',')
                cargList = [x.strip(' ') for x in cargList]
                for arg in cargList:
                    e_cargList.append(arg.split('.')[len(arg.split('.'))-1])

                #extract args from groundtruth
                gargs = p[6][p[6].index('(') + 1: p[6].index(')')]
                # gargList = gargs.split(',')
                e_gargList = gargs.split(',')
                e_gargList = [x.strip(' ') for x in e_gargList]

                #compare arguments
                if set(e_cargList) == set(e_gargList):
                    found = True
                    print("Found out positive datapoint:", p[6])
            
            if not found:
                positive_position = positive_position + 1
            
            # can_pair = [p[5], can[1][: can[1].index('(') + 1] + ','.join(e_cargList) + ')']
            can_pair = [
                '.'.join(p[5].split('.')[2:]),
                '.'.join( (can[1][: can[1].index('(') + 1] + ','.join(e_cargList) + ')').split('.')[2:] )
                ]
            can_pairs.append(can_pair)


    #if the json has the positive pair, remove 
    if found:
        can_pairs.pop(positive_position)

    
    if not found:
        print("Found no positive datapoint!")
    else:
        # posPair = [p[5], p[6]]
        posPair = [
            '.'.join(p[5].split('.')[2:]),
            '.'.join(p[6].split('.')[2:])
            ]
        pos_pairs.append(divider.join(posPair))

        #rank negative pairs by levenshtein distance
        dist = []
        for np in can_pairs:
            if '$' not in np[1]:
                dist.append([np[1], distance(np[1], posPair[1])])

        sortedDist = sorted(dist, key=lambda npd: npd[1])

        for i in range(max(int(len(sortedDist)/1000), 10)):
            neg_pairs.append(posPair[0] + divider + sortedDist[i][0])


    pcnt = pcnt + 1

Start time: 2022-04-03 09:54:43.004992
0 / 504 Processing: mockito-core-2.2.17.jar.json
Searching: org.mockito.ArgumentMatchers.matches(String)
Found out positive datapoint: org.mockito.ArgumentMatchers.matches(String)
1 / 504 Processing: mockito-core-2.2.17.jar.json
Searching: org.mockito.ArgumentMatchers.anyString()
Found out positive datapoint: org.mockito.ArgumentMatchers.anyString()
2 / 504 Processing: mockito-core-3.5.10.jar.json
Searching: org.mockito.ArgumentMatchers.anyString()
Found out positive datapoint: org.mockito.ArgumentMatchers.anyString()
3 / 504 Processing: mockito-core-3.5.10.jar.json
Searching: org.mockito.ArgumentMatchers.any()
Found out positive datapoint: org.mockito.ArgumentMatchers.any()
4 / 504 Processing: mockito-core-3.5.10.jar.json
Searching: org.mockito.ArgumentMatchers.anyObject()
Found out positive datapoint: org.mockito.ArgumentMatchers.anyObject()
5 / 504 Processing: mockito-core-3.5.10.jar.json
Searching: org.mockito.ArgumentMatchers.anyBoolean()
Fou

In [7]:
try:
    os.remove("G:\My Drive\Term 4.2\IS706 - Software Mining and Analysis\Project\\dataset\\training\\raw\\positive.txt")
except:
    print("File not found")
try:
    os.remove("G:\My Drive\Term 4.2\IS706 - Software Mining and Analysis\Project\\dataset\\training\\raw\\negative.txt")
except:
    print("File not found")
try:
    os.remove("G:\My Drive\Term 4.2\IS706 - Software Mining and Analysis\Project\\dataset\\training\\cleaned\\yes.txt")
except:
    print("File not found")
try:
    os.remove("G:\My Drive\Term 4.2\IS706 - Software Mining and Analysis\Project\\dataset\\training\\cleaned\\no.txt")
except:
    print("File not found")

try:
    os.remove("G:\My Drive\Term 4.2\IS706 - Software Mining and Analysis\Project\\dataset\\validation\\raw\\positive.txt")
except:
    print("File not found")
try:
    os.remove("G:\My Drive\Term 4.2\IS706 - Software Mining and Analysis\Project\\dataset\\validation\\raw\\negative.txt")
except:
    print("File not found")
try:
    os.remove("G:\My Drive\Term 4.2\IS706 - Software Mining and Analysis\Project\\dataset\\validation\\cleaned\\yes.txt")
except:
    print("File not found")
try:
    os.remove("G:\My Drive\Term 4.2\IS706 - Software Mining and Analysis\Project\\dataset\\validation\\cleaned\\no.txt")
except:
    print("File not found")

File not found
File not found
File not found
File not found
File not found
File not found
File not found
File not found


In [4]:
print("Extracted size:", len(pos_pairs), len(neg_pairs))

Extracted size: 404 5125


<h2>Write Datapoints to file for ML

In [5]:
pos_pairs_to_pop = list(set(pos_pairs))
neg_pairs_to_pop = list(set(neg_pairs))

print("Dataset size:", len(pos_pairs_to_pop), len(neg_pairs_to_pop))#SEEDDD

Dataset size: 282 2939


In [6]:
try:
    os.remove('G:\My Drive\Term 4.2\IS706 - Software Mining and Analysis\Project\\dataset-other\\yes.txt')
except:
    print("File not found")
try:
    os.remove('G:\My Drive\Term 4.2\IS706 - Software Mining and Analysis\Project\\dataset-other\\no.txt')
except:
    print("File not found")

with open('G:\My Drive\Term 4.2\IS706 - Software Mining and Analysis\Project\\dataset-other\\positive.txt', 'a', newline='') as f_object: 
        
    writer_object = writer(f_object)
        
    for pair in pos_pairs_to_pop:

        writer_object.writerow([pair.replace('"', '')]) 

    f_object.close()

with open('G:\My Drive\Term 4.2\IS706 - Software Mining and Analysis\Project\\dataset-other\\negative.txt', 'a', newline='') as f_object: 
        
    writer_object = writer(f_object)

    for pair in neg_pairs_to_pop:

        writer_object.writerow([pair.replace('"', '')]) 

    f_object.close()

with open('G:\My Drive\Term 4.2\IS706 - Software Mining and Analysis\Project\\dataset-other\\positive.txt', 'r') as f, open('G:\My Drive\Term 4.2\IS706 - Software Mining and Analysis\Project\\dataset-other\\yes.txt', 'w') as fo:
    for line in f:
        fo.write(line.replace('"', '').replace("'", ""))

with open('G:\My Drive\Term 4.2\IS706 - Software Mining and Analysis\Project\\dataset-other\\negative.txt', 'r') as f, open('G:\My Drive\Term 4.2\IS706 - Software Mining and Analysis\Project\\dataset-other\\no.txt', 'w') as fo:
    for line in f:
        fo.write(line.replace('"', '').replace("'", ""))

try:
    os.remove('G:\My Drive\Term 4.2\IS706 - Software Mining and Analysis\Project\\dataset-other\\positive.txt')
except:
    print("File not found")
try:
    os.remove('G:\My Drive\Term 4.2\IS706 - Software Mining and Analysis\Project\\dataset-other\\negative.txt')
except:
    print("File not found")



<h2>Oversampling Positive Datapoints

In [57]:
pos_pairs_to_pop = list(set(pos_pairs))
neg_pairs_to_pop = list(set(neg_pairs))

print("To divide size:", len(pos_pairs_to_pop), len(neg_pairs_to_pop))

To divide size: 282 2939


In [35]:
training_positives = []
training_negatives = []

validation_positives = []
validation_negatives = []

divide = 1/7

for i in range(0, int(len(neg_pairs_to_pop)*divide)):
    validation_negatives.append(neg_pairs_to_pop.pop(random.randrange(0, len(neg_pairs_to_pop))))
training_negatives = neg_pairs_to_pop

posPopped = 0
for i in range(0, int(len(pos_pairs_to_pop)*divide)):
    validation_positives.append(pos_pairs_to_pop.pop(random.randrange(0, len(pos_pairs_to_pop))))
    posPopped = posPopped + 1
print("Chosen", posPopped, "for validation")
training_positives = numpy.random.choice(pos_pairs_to_pop, len(training_negatives))

Chosen 7 for validation


In [32]:
print("Extracted size:", len(pos_pairs), len(neg_pairs))
print("Training size:", len(training_positives), len(training_negatives))
print("Validation size:", len(validation_positives), len(validation_negatives))

Extracted size: 339 4475
Training size: 2028 2028
Validation size: 30 337


In [ ]:
with open('G:\My Drive\Term 4.2\IS706 - Software Mining and Analysis\Project\\dataset\\training\\raw\\positive.txt', 'a', newline='') as f_object: 
        
    writer_object = writer(f_object)
        
    for pair in training_positives:

        writer_object.writerow([pair.replace('"', '')]) 

    f_object.close()

with open('G:\My Drive\Term 4.2\IS706 - Software Mining and Analysis\Project\\dataset\\training\\raw\\negative.txt', 'a', newline='') as f_object: 
        
    writer_object = writer(f_object)

    for pair in training_negatives:

        writer_object.writerow([pair.replace('"', '')]) 

    f_object.close()

with open('G:\My Drive\Term 4.2\IS706 - Software Mining and Analysis\Project\\dataset\\training\\raw\\positive.txt', 'r') as f, open('G:\My Drive\Term 4.2\IS706 - Software Mining and Analysis\Project\\dataset\\training\\cleaned\\yes.txt', 'w') as fo:
    for line in f:
        fo.write(line.replace('"', '').replace("'", ""))

with open('G:\My Drive\Term 4.2\IS706 - Software Mining and Analysis\Project\\dataset\\training\\raw\\negative.txt', 'r') as f, open('G:\My Drive\Term 4.2\IS706 - Software Mining and Analysis\Project\\dataset\\training\\cleaned\\no.txt', 'w') as fo:
    for line in f:
        fo.write(line.replace('"', '').replace("'", ""))

#validation

with open('G:\My Drive\Term 4.2\IS706 - Software Mining and Analysis\Project\\dataset\\validation\\raw\\positive.txt', 'a', newline='') as f_object: 
        
    writer_object = writer(f_object)
        
    for pair in validation_positives:

        writer_object.writerow([pair.replace('"', '')]) 

    f_object.close()

with open('G:\My Drive\Term 4.2\IS706 - Software Mining and Analysis\Project\\dataset\\validation\\raw\\negative.txt', 'a', newline='') as f_object: 
    
    writer_object = writer(f_object)

    for pair in validation_negatives:

        writer_object.writerow([pair.replace('"', '')]) 

    f_object.close()

with open('G:\My Drive\Term 4.2\IS706 - Software Mining and Analysis\Project\\dataset\\validation\\raw\\positive.txt', 'r') as f, open('G:\My Drive\Term 4.2\IS706 - Software Mining and Analysis\Project\\dataset\\validation\\cleaned\\yes.txt', 'w') as fo:
    for line in f:
        fo.write(line.replace('"', '').replace("'", ""))

with open('G:\My Drive\Term 4.2\IS706 - Software Mining and Analysis\Project\\dataset\\validation\\raw\\negative.txt', 'r') as f, open('G:\My Drive\Term 4.2\IS706 - Software Mining and Analysis\Project\\dataset\\validation\\cleaned\\no.txt', 'w') as fo:
    for line in f:
        fo.write(line.replace('"', '').replace("'", ""))